In [1]:
import os
import csv
import math
import itertools
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from scipy import sparse
import networkx as nx

from mbi import (
    Dataset,
    FactoredInference,
    Domain,
    LocalInference,
    MixtureInference,
    PublicInference,
)

In [2]:
data = Dataset.load("./data/diabetes_processed.csv", "./data/diabetes_processed.json")
domain = data.domain
total = data.df.shape[0]

In [3]:
# adapted from https://github.com/ryan112358/private-pgm/blob/master/examples/adult_example.py

cliques = [('DiabetesPedigreeFunction', 'Outcome'), ('Insulin', 'DiabetesPedigreeFunction'), ('BMI', 'DiabetesPedigreeFunction'), ('Pregnancies', 'DiabetesPedigreeFunction'), ('DiabetesPedigreeFunction', 'Age'), ('Glucose', 'DiabetesPedigreeFunction'), ('SkinThickness', 'DiabetesPedigreeFunction'), ('BloodPressure', 'DiabetesPedigreeFunction')]

# spend half of privacy budget to measure all 1 way marginals
np.random.seed(0)

epsilon = 1.0
epsilon_split = epsilon / (len(data.domain) + len(cliques))
sigma = 2.0 / epsilon_split

measurements = []
for col in data.domain:
    x = data.project(col).datavector()
    y = x + np.random.laplace(loc=0, scale=sigma, size=x.size)
    I = sparse.eye(x.size)
    measurements.append((I, y, sigma, (col,)))

# spend half of privacy budget to measure some more 2 and 3 way marginals
for cl in cliques:
    x = data.project(cl).datavector()
    y = x + np.random.laplace(loc=0, scale=sigma, size=x.size)
    I = sparse.eye(x.size)
    measurements.append((I, y, sigma, cl))

In [ ]:
# THIS TAKES <1 MINUTE
engine = FactoredInference(domain, log=True, iters=500)
model = engine.estimate(measurements, total=total)

Total clique size: 395505
iteration		time		l1_loss		l2_loss		feasibility
0.00		0.00		397339.96		397697.62		0.00
50.00		3.01		397061.58		396754.71		0.00
100.00		6.23		397056.83		396706.71		0.00
150.00		9.83		397056.32		396703.41		0.00
200.00		13.23		397056.19		396702.66		0.00
250.00		16.71		397056.15		396702.40		0.00
300.00		20.36		397056.16		396702.29		0.00
350.00		24.27		397056.18		396702.23		0.00
400.00		29.44		397056.19		396702.20		0.00
450.00		37.94		397056.20		396702.18		0.00


In [ ]:
# Save the model to a file
# this is about 6MB
with open('./model/diabetes_synth.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# this takes around <1 second
synth = model.synthetic_data(rows=800)
sdf = synth.df
sdf

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,7,105,24,46,143,83,323,22,0
1,4,14,11,21,127,13,432,25,0
2,2,130,11,21,132,184,196,54,0
3,2,105,2,10,77,71,97,22,0
4,6,111,46,45,36,109,410,21,0
...,...,...,...,...,...,...,...,...,...
795,3,47,15,27,130,13,17,29,1
796,0,53,23,10,77,125,390,23,0
797,2,130,11,21,132,160,196,21,0
798,5,55,27,21,25,160,429,62,0
